<a href="https://colab.research.google.com/github/kawai-katsumi-lang/redesigned-octo-garbanzo/blob/main/%E9%9F%B3%E5%A3%B0%E5%9F%8B%E3%82%81%E8%BE%BC%E3%81%BF%E3%83%84%E3%83%BC%E3%83%AB_%E5%BC%B7%E5%8A%9B%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =================================================================
# データドリブン経営プレゼン用: 音声埋め込みツール (強力置換版)
# コードの微妙な違いを無視して、強制的に埋め込み対応にします。
# =================================================================

import os
import base64
import re
from google.colab import files

def embed_audio():
    html_content = ""
    html_filename = ""
    audio_data = {}

    # ---------------------------------------------------------
    # ステップ 1: HTMLファイルのアップロード
    # ---------------------------------------------------------
    print("🚀 【ステップ 1/2】")
    print("   ベースとなる HTMLファイル (.html) をアップロードしてください。")
    print("-" * 50)

    uploaded_html = files.upload()

    if not uploaded_html:
        print("❌ HTMLファイルが選択されませんでした。")
        return

    for filename, content in uploaded_html.items():
        if filename.endswith(".html"):
            html_content = content.decode('utf-8')
            html_filename = filename
            print(f"✅ HTMLを受け付けました: {filename}")
            break

    if not html_content:
        print("❌ .htmlファイルが見つかりませんでした。")
        return

    # ---------------------------------------------------------
    # ステップ 2: 音声ファイルのアップロード
    # ---------------------------------------------------------
    print("\n🚀 【ステップ 2/2】")
    print("   音声ファイル (.mp3) をすべてアップロードしてください。")
    print("-" * 50)

    uploaded_audio = files.upload()

    print("\n📂 音声ファイルを処理中...")
    for filename, content in uploaded_audio.items():
        if filename.lower().endswith(".mp3"):
            b64_str = base64.b64encode(content).decode('utf-8')
            mime = "data:audio/mp3;base64," + b64_str
            audio_data[filename] = mime
            print(f"   🎵 エンコード完了: {filename}")

    # ---------------------------------------------------------
    # 強力な置換処理
    # ---------------------------------------------------------

    # 1. 埋め込みデータの定義を作成
    js_audio_obj = "const EMBEDDED_AUDIO = {\n"
    for name, data in audio_data.items():
        js_audio_obj += f'    "{name}": "{data}",\n'
    js_audio_obj += "};\n"

    # 2. データの注入 (DOMContentLoadedの直前)
    if 'document.addEventListener(\'DOMContentLoaded\', () => {' in html_content:
        html_content = html_content.replace(
            'document.addEventListener(\'DOMContentLoaded\', () => {',
            f"{js_audio_obj}\ndocument.addEventListener('DOMContentLoaded', () => {{"
        )
    else:
        # 見つからない場合はscriptタグの先頭に
        html_content = html_content.replace('<script>', f'<script>\n{js_audio_obj}')

    # 3. playNarration関数のロジックを「範囲指定」で強制書き換え
    # "const slideNumber" から "audioPlayer.src =" の直前までをターゲットにする
    # これにより、途中の変数がどう書かれていようと無視して上書きする

    start_marker = r'const\s+slideNumber\s*=\s*String\(index\s*\+\s*1\)\.padStart'
    # audioPlayer.src への代入、または URL生成部分までを広くマッチさせる
    end_marker = r'audioPlayer\.src\s*=\s*audioSrc;'

    # 新しいロジック (埋め込み対応版)
    new_logic_block = """
        const slideNumber = String(index + 1).padStart(2, '0');
        const audioFilename = `slide_${slideNumber}.mp3`;

        let audioSrc;
        // 埋め込みデータがあるかチェック
        if (typeof EMBEDDED_AUDIO !== 'undefined' && EMBEDDED_AUDIO[audioFilename]) {
            audioSrc = EMBEDDED_AUDIO[audioFilename];
        } else {
            // なければ通常のフォルダ参照 (変数 AUDIO_FOLDER がある前提、なければ 'audio'固定)
            const folder = (typeof AUDIO_FOLDER !== 'undefined') ? AUDIO_FOLDER : 'audio';
            audioSrc = `${folder}/${audioFilename}`;
        }

        // Base64の場合はURLオブジェクト化をスキップ
        let resolvedSrc = audioSrc;
        if (!audioSrc.startsWith('data:')) {
            resolvedSrc = new URL(audioSrc, window.location.href).href;
        }

        if (audioPlayer.src === resolvedSrc && !audioPlayer.paused) {
            return;
        }

        audioPlayer.src = audioSrc;
    """

    # 正規表現でブロック置換を実行
    # const slideNumber ... (中略) ... audioPlayer.src = audioSrc; までを置換
    pattern = rf'({start_marker}[\s\S]*?){end_marker}'

    match = re.search(pattern, html_content)
    if match:
        # マッチした全体を、新しいロジックに置き換える
        html_content = re.sub(pattern, new_logic_block, html_content)
        print("\n✅ HTMLコードの書き換えに成功しました（強制置換モード）")
    else:
        print("\n⚠️ 警告: コードの書き換え箇所が見つかりませんでした。")
        print("   playNarration関数内の `const slideNumber = ...` から `audioPlayer.src = ...` の形が変わっている可能性があります。")

    # 4. 保存とダウンロード
    output_filename = "complete_presentation_final.html"
    with open(output_filename, "w", encoding='utf-8') as f:
        f.write(html_content)

    print(f"\n🎉 生成完了: {output_filename}")
    files.download(output_filename)

if __name__ == "__main__":
    embed_audio()

🚀 【ステップ 1/2】
   ベースとなる HTMLファイル (.html) をアップロードしてください。
--------------------------------------------------


Saving データドリブン経営_20251203.html to データドリブン経営_20251203.html
✅ HTMLを受け付けました: データドリブン経営_20251203.html

🚀 【ステップ 2/2】
   音声ファイル (.mp3) をすべてアップロードしてください。
--------------------------------------------------


Saving slide_01.mp3 to slide_01.mp3
Saving slide_02.mp3 to slide_02.mp3
Saving slide_03.mp3 to slide_03.mp3
Saving slide_04.mp3 to slide_04.mp3
Saving slide_05.mp3 to slide_05.mp3
Saving slide_06.mp3 to slide_06.mp3
Saving slide_07.mp3 to slide_07.mp3
Saving slide_08.mp3 to slide_08.mp3
Saving slide_09.mp3 to slide_09.mp3
Saving slide_10.mp3 to slide_10.mp3
Saving slide_11.mp3 to slide_11.mp3
Saving slide_12.mp3 to slide_12.mp3
Saving slide_13.mp3 to slide_13.mp3
Saving slide_14.mp3 to slide_14.mp3
Saving slide_15.mp3 to slide_15.mp3
Saving slide_16.mp3 to slide_16.mp3
Saving slide_17.mp3 to slide_17.mp3
Saving slide_18.mp3 to slide_18.mp3
Saving slide_19.mp3 to slide_19.mp3

📂 音声ファイルを処理中...
   🎵 エンコード完了: slide_01.mp3
   🎵 エンコード完了: slide_02.mp3
   🎵 エンコード完了: slide_03.mp3
   🎵 エンコード完了: slide_04.mp3
   🎵 エンコード完了: slide_05.mp3
   🎵 エンコード完了: slide_06.mp3
   🎵 エンコード完了: slide_07.mp3
   🎵 エンコード完了: slide_08.mp3
   🎵 エンコード完了: slide_09.mp3
   🎵 エンコード完了: slide_10.mp3
   🎵 エンコード完了: slide_11.mp3
  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>